# Construcción variables Activo

In [1]:
import pandas as pd 
import numpy as np 

pd.set_option('max_columns',None)
pd.set_option('max_rows',None)

from datetime import datetime, timedelta 
from dateutil import relativedelta
import gc

In [2]:
import s3fs
import pyarrow as pa
import pyarrow.parquet as pq

def spark_read_parquet(s3_url: str, **args):
    fs = s3fs.S3FileSystem()
    # Leyendo base
    dataset = pq.ParquetDataset(s3_url, filesystem=fs)
    table = dataset.read()
    dataframe = table.to_pandas()

    del dataset, table

    return dataframe

In [3]:
fs = s3fs.S3FileSystem()

In [4]:
var_finales = [
'amortizacion_min_LB',# activos
'cuota_max_LB_SMLV',# activos

'num_lib_solicitadas',# libranzas
'prom_monto_novado', # libranzas
'prom_n_cuotas', # libranzas


'pasv_antig_total', # pasivos
'pasv_saldo_ca_1mes_atras',# pasivos

'sum_tx_linea_ahr',# TX Pasivo
'dias_desde_ult_tx_ahr',# tx_pasivos
'num_tx_ult_1mes_ahr',# tx_pasivos

'valor_cuotas_codeudores_smlv', # Buro
'valor_utilizado_smlv',# Buro
'numero_obligaciones_activasdif',# buro
'porcentaje_utilizacion',# buro
'quanto_mod',# buro

'num_no_aceptado',# campañas
]

In [5]:
'amortizacion_min_LB',
'cuota_max_LB_SMLV',

'numero_obligaciones_activasdif',#
'porcentaje_utilizacion',#
'quanto_mod',#
'valor_cuotas_codeudores_smlv',#
'valor_utilizado_smlv',#

'num_lib_solicitadas',#
'prom_monto_novado',#
'prom_n_cuotas',#

('prom_n_cuotas',)

In [6]:
var_finales = [
#
#


'num_no_aceptado',#
    
'dias_desde_ult_tx_ahr',#

'pasv_antig_total',#
'pasv_saldo_ca_1mes_atras',#

'sum_tx_linea_ahr',#
'num_tx_ult_1mes_ahr',#
    
]

In [7]:
"""
MODULE: processing_activos
This script extracts activos information from the activos file
Steps:
1. Get activos file
2. Get the needed columns and rows from activos file including the needed historical period
3. Create additional columns
"""


def process_activos(input_paths_activos):
    """Pending docstring"""
    # 1. Get activos file
    activos = get_activos_file(input_paths_activos)
    print('get_activos_file')
    # 2. Deletes duplicates and casts variables
    activos = get_activos_info(activos)
    print('get_activo_info')
    # 3. Creates aux variables requeried for the final explanatory variables
    activos = create_activos_info(activos)
    print('create_activos_info')
    # 4. Aggregate and produce client-level output data frame
    activos_cli_fec = get_activos_cliente(activos)
    print('get_activos_cliente')
    # 5. Produce client-level output data frame with explanatory variables
    activos_final = create_activos_df(activos, activos_cli_fec)
    print('create_activos_df')
    print('activos_df created successfully')

    return activos_final


def get_activos_file(input_path):
    """ Gets the input_path to the activos file, drops some not useful columns
     and outputs a DataFrame
    :param input_path to activos file location
    :return: DataFrame
    """

    activos_file_path = input_path
    activos = spark_read_parquet(activos_file_path)

    activos.rename(columns=lambda x: x.lower(), inplace=True)

    if activos['id_cliente'].isnull().sum() > 0:
        activos = activos.loc[activos['id_cliente'].notnull()]

    str_date = input_path[-6:]
    activos['fecha_activos'] = pd.to_datetime(str_date, format='%Y%m').strftime('%Y%m')

    # cast nroident
    activos['id_cliente'] = activos['id_cliente'].fillna('-999').astype(np.int64)
    activos = activos[
        ['id_cliente', 'nrooblig', 'fecha_activos', 'dias_morak', 'lineacredi', 'tipo_producto', 'sal_capita',
         'vlr_cuota', 'vlr_desemb']]

    return activos


def get_activos_info(activos):
    """Deletes duplicates and casts variables
    :param activos to activos file location
    :return: DataFrame
    """

    # delete negative nroident
    activos = activos[(activos['id_cliente'] > 0)]

    # delete duplicates
    activos = activos.sort_values(['id_cliente', 'fecha_activos'])
    activos = activos.drop_duplicates(subset=['id_cliente', 'nrooblig', 'fecha_activos'], keep='first')

    # casting variables
    activos['fecha_activos'] = activos['fecha_activos'].astype(int)

    return activos


def create_activos_info(activos):
    """Creates aux variables requeried for the final explanatory variables
    :param activos to activos file location
    :return: DataFrame
    """
    # Creates salario minimo mensual vigente
    activos['ano_po'] = activos['fecha_activos'].astype(str).str[0:4].astype(int)
    activos['smlv'] = np.where(activos['ano_po'] == 2018, 781242,
                               np.where(activos['ano_po'] == 2019, 828116,
                                        np.where(activos['ano_po'] == 2020, 877803,
                                                 np.where(activos['ano_po'] == 2021, 908526,
                                                 np.nan))))

    # Creates active credits flag
    #activos['cuenta_vigente'] = np.where(activos['estado'] == 'ACTIVO', 1, 0)
    activos.loc[:, 'cuenta_vigente'] = 1

    # Creates tipo_prod
    activos['tipo_prod'] = np.where((activos['lineacredi'] == 61) & (activos['tipo_producto'] == 'TC'), 'TC_T',
                                    np.where((activos['lineacredi'] == 62) & (activos['tipo_producto'] == 'TC'), 'TC_X',
                                             activos['tipo_producto']))
   
    #Variable amortizacion a nivel obligación
    activos['amortizacion']=np.where((activos['vlr_desemb']> 0) & (activos['sal_capita'] >= 0),(activos['vlr_desemb']-activos['sal_capita'])/activos['vlr_desemb'],
    np.where((activos['vlr_desemb'] > 0) & np.isnan(activos['sal_capita']),1,np.nan))

    return activos


def get_activos_cliente(activos):
    """Takes a product-level data frame and returns a client-level data frame,
    :param activos complete activos DataFrame
    :return: DataFrame, client-level
    """

    # Creates data base at 'nroident','fecha_activos' level.
    activos_cli_fec = pd.DataFrame(activos.groupby(['id_cliente', 'fecha_activos'])[['smlv']].max())

    return activos_cli_fec


def create_activos_df(activos, activos_cli_fec):
    """Creates client-level final variables
    :param activos complete activos DataFrame
    :param activos_cli_fec containing activos relevant rows and columns and activos_cli_fec client level data base
    :return: DataFrame
    """

    # Numero de obligaciones libranza al dia
    activos_cli_fec['amortizacion_min_lb'] = activos[
        (activos['cuenta_vigente'] == 1) & (activos['tipo_producto'] == 'LB')].groupby(
        ['id_cliente','fecha_activos'])[['amortizacion']].min()


    # Saldo y cuota
    activos_cli_fec['cuota_max_lb']  = activos[
        (activos['tipo_producto'] == 'LB') & (activos['cuenta_vigente'] == 1)].groupby(
        ['id_cliente','fecha_activos'])[['vlr_cuota']].max()

    
   
    # Saldo y cuota / smlv
    activos_cli_fec['cuota_max_lb_smlv'] = activos_cli_fec['cuota_max_lb'] / activos_cli_fec['smlv']

    activos_cli_fec = activos_cli_fec.reset_index()
    activos_out = activos_cli_fec[
        ['id_cliente', 'fecha_activos', 'amortizacion_min_lb', 'cuota_max_lb_smlv']]
    print('Shape inicial:',activos_out.shape[0])
    activos_out = activos_out.loc[activos_out['amortizacion_min_lb'].notnull()]
    print('Shape final:',activos_out.shape[0])
   
    return activos_out


In [7]:
period = '202104'
path = 's3://data-bpop-dev-sandbox/estandarizado/productos/activo-csv/productos_activo-csv_sarc_M'
file_name = path + period

data_activos = process_activos(file_name)
print(data_activos.shape)
data_activos.head()

/home/jupyter/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


get_activos_file
get_activo_info
create_activos_info
get_activos_cliente
Shape inicial: 467620
Shape final: 331863
create_activos_df
activos_df created successfully
(331863, 4)


,id_cliente,fecha_activos,amortizacion_min_lb,cuota_max_lb_smlv
3,101052314032556001,202104,0.046968,1.231402
7,101055260683554401,202104,0.016047,0.668039
8,101055266367157902,202104,0.184722,0.723335
9,101055266603150201,202104,0.386253,0.168098
10,101055266686923002,202104,0.038940,0.831396


In [8]:
data_activos.count()

id_cliente             331863
fecha_activos          331863
amortizacion_min_lb    331863
cuota_max_lb_smlv      331863
dtype: int64

In [9]:
path_out = 's3://adl-refined-dev-popular/parquet/TC_adquisicion/base_activos_M'
file_name_out = path_out + period

data_activos.to_parquet(file_name_out,engine='pyarrow', index=False)
